In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama
import os

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

dir = "pdf/"
metadatas = []
chunks = []  
chunk_size = 432
chunk_overlap = 8

In [3]:
for arq in os.listdir(dir):
    
    start_index = 0
    text_join = ""
    
    metadatas.append({"name": arq})
    print(arq)
    text = PdfReader(dir+arq)
    text_join = ''.join(page.extract_text() for page in text.pages)

    for i in range(0, len(text_join), chunk_size):
        start = i
        end = i + chunk_size
        if start != 0:
            start = start - chunk_overlap
            end =  end + chunk_overlap
        chunks.append(text_join[start:end])
        metadatas.append({"name": arq, "page": i})

embeddings_db = embeddings.encode(chunks)

Livro de Arquitetura e organização de computadores.pdf
Learning LangChain-O'Reilly Media (2024).pdf
JavaBasico.pdf


In [4]:
for i, d in enumerate(chunks):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadatas[i]
    )

In [ ]:
while True:

    query = []

    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=10,
        )

    metadata = results['metadatas'][0]
    data = results['documents'][0]
    distance = results['distances'] 
    
    print(distance)
    print(data)
          
    for k in range(10):
        if distance[0][k] < 0.74:
            query.append(data[k])

    prompt_template = f"""You are a chatbot specialized in generating quizzes. 
                        Generate a quiz with 5 questions in Portuguese in the following context: {query} with the following format: question, options and explanation of the right answer"""

    if len(query) == 0:
            prompt_template = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    
    output = ollama.generate(
            model="gemma",
            prompt=prompt_template
            )
    print("\n")
    output = output['response']
    print(output)
    print("\n")
    

Pergunte algo memória ram


[[0.5562481880187988, 0.6207898855209351, 0.6782195568084717, 0.6851325631141663, 0.7022801637649536, 0.7119332551956177, 0.7519334554672241, 0.7666471004486084, 0.7671561241149902, 0.7728975415229797]]
['RAM) é uma forma de memória de acesso aleatório usada para a memória cache; veja o Capítulo 5.\n15 A memória flash consiste em uma forma versátil de memória usada tanto por microcontroladores como por memória externa; esse assunto é discutido no Capítulo 6.\nBOKK_STALLINGS.indb   31 01/09/17   09:15Arquitetura e organização de computadores32\n}}Gerenciamento de energia: administra os vários modos de operação do processador e de periféricos em \nbai', '8\nRAM não volátil dentro da hierarquia da memória.\nSRAM\nSTT-RAM\nPCRAM\nReRAMDesempenho\ne durabilidade\naumentados\nCusto decrescente\npor bit, capacidade\nou densidade crescentesDRAM\nFLASH NAND \nDISCO RÍGIDO \nBOKK_STALLINGS.indb   156 01/09/17   09:15Capítulo 5 } Memória interna  \n157\nAo longo do tempo, cada uma dessas tecnolog

Pergunte algo langchain


[[0.708989679813385, 0.785416305065155, 0.8077477216720581, 0.8599618673324585, 0.8600448369979858, 0.867042064666748, 0.8695042729377747, 0.8980491161346436, 0.9132727384567261, 0.9156815409660339]]
[' for sharing with\nteammates.\nLangChain contains many integrations with third-party\nservices (like Google Sheets, W olfram Alpha, Zapier , just to\nname a few) exposed as tools , which is a standard interface\nfor functions to be used in the tool-calling technique.\nFor RAG , LangChain provides integrations with the major\nembedding models  (language models designed to output a\nnumeric representation, the embedding , of the meaning ofa sentence, par', 'nally , LangChain provides the tools to compose these\nbuilding blocks into cohesive applications. Chapters 1\nthrough 6 talk more about this.In addition to this library , LangChain provides LangSmith , a\nplatform to help debug, test, deploy , and monitor AI\nworkﬂows, and LangServe , a platform that makes it easier\nto deploy a LangCh